In [1]:
import requests
import torch
from PIL import Image
from transformers import MllamaForConditionalGeneration, AutoProcessor
from huggingface_hub import login

if torch.cuda.is_available():
    print("NVIDIA GPU is available.")
    device = torch.device("cuda")  # Use the GPU
    print(f"Using device: {torch.cuda.get_device_name(device)}")
else:
    print("NVIDIA GPU is not available.")
    device = torch.device("cpu")  # Fallback to CPU
    print("Using CPU.")

/home/thanh.ho/.conda/envs/taco/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NVIDIA GPU is available.
Using device: NVIDIA A100-SXM4-40GB


In [2]:
access_token = ''
# Log in with your Hugging Face token
login(token=access_token)

model_id = "meta-llama/Llama-3.2-11B-Vision"

model = MllamaForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    token = access_token
)
processor = AutoProcessor.from_pretrained(model_id)

#url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/0052a70beed5bf71b92610a43a52df6d286cd5f3/diffusers/rabbit.jpg"
#image = Image.open(requests.get(url, stream=True).raw)

The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
Loading checkpoint shards: 100%|██████████| 5/5 [00:47<00:00,  9.42s/it]


In [3]:
! pip install 'accelerate>=0.26.0'

In [15]:
SYSTEM_PROMPT_INTENT_DETECTION = """
The current time is 2024-01-15 03:47:43.

You are a helpful assistant. Your task is to detect the user's intent and provide a response in the form of a JSON object complete with the following keys:
1. 'patient_id': A string representing the ID of the patient the user is inquiring about. If patient ID is not provided, leave it empty.
2. 'list_date': A list of dates for which data needs to be retrieved to answer the user's question in format of yyyy-mm-dd. Leave the list empty if the user asks for data right now.
3. 'list_time':  A list of times for which data needs to be retrieved to answer the user's question in format of hh:mm:ss. The system saves data in 30-minute period like 00:00:00 and 00:30:00. Leave the list empty if the user asks for data right now.
4. 'vital_sign': A list of vital signs that the user is asking for. Here are the available vital sign: heart_rate, systolic_pressure, diastolic_pressure, respiratory_rate, body_temperature, oxygen_saturation
5. 'is_plot': A Boolean value indicating whether the system needs to generate a plot to answer the question more clearly (when the number of data points is too large) or if the user has requested a plot.
6. 'recognition': A Boolean value indicating whether the user is asking for activity or emotion recognition of the patient. Set to true if the user is asking for this information, otherwise false.
7. 'is_image': A Boolean value indicating whether the user is asking to show an image of the patient.

If the user provides the time in both AM and PM formats, you can still interpret it in the 'hh:mm:ss' format.
If the user asks for sessions during the day, please use the following information to fill in list_time: Morning is from 05:00:00 (5am) to 12:00:00 (12pm), Afternoon is from 12:00:00 (12pm) to 17:00:00 (5pm), Evening is from 17:00:00 (5pm) to 21:00:00 (9pm), Night is from 21:00:00 (9pm) to 04:00:00 (4am) the following day.
It's important to remember that health status consists of vital signs.
When the user asks for data from the last a number of hours, today should also be included in the list_date.
When the user asks for data last morning, last afternoon, last evening, last night, fill the previous day and today in list_date.
When users ask questions related to time periods such as last month, last hours, last days, or last week, try your best to fill in list_date and list_time with full dates and times within the specified range.

Provide me the respiratory rate and systolic pressure of the patient 05419 in the last 19 days.
"""

In [28]:
prompt = """<|start_header_id|>system<|end_header_id|>
The current time is 2024-01-15 03:47:43.

You are a helpful assistant. Your task is to detect the user's intent and provide a response in the form of a JSON object complete with the following keys:
1. 'patient_id': A string representing the ID of the patient the user is inquiring about. If patient ID is not provided, leave it empty.
2. 'list_date': A list of dates for which data needs to be retrieved to answer the user's question in format of yyyy-mm-dd. Leave the list empty if the user asks for data right now.
3. 'list_time':  A list of times for which data needs to be retrieved to answer the user's question in format of hh:mm:ss. The system saves data in 30-minute period like 00:00:00 and 00:30:00. Leave the list empty if the user asks for data right now.
4. 'vital_sign': A list of vital signs that the user is asking for. Here are the available vital sign: heart_rate, systolic_pressure, diastolic_pressure, respiratory_rate, body_temperature, oxygen_saturation
5. 'is_plot': A Boolean value indicating whether the system needs to generate a plot to answer the question more clearly (when the number of data points is too large) or if the user has requested a plot.
6. 'recognition': A Boolean value indicating whether the user is asking for activity or emotion recognition of the patient. Set to true if the user is asking for this information, otherwise false.
7. 'is_image': A Boolean value indicating whether the user is asking to show an image of the patient.

If the user provides the time in both AM and PM formats, you can still interpret it in the 'hh:mm:ss' format.
If the user asks for sessions during the day, please use the following information to fill in list_time: Morning is from 05:00:00 (5am) to 12:00:00 (12pm), Afternoon is from 12:00:00 (12pm) to 17:00:00 (5pm), Evening is from 17:00:00 (5pm) to 21:00:00 (9pm), Night is from 21:00:00 (9pm) to 04:00:00 (4am) the following day.
It's important to remember that health status consists of vital signs.
When the user asks for data from the last a number of hours, today should also be included in the list_date.
When the user asks for data last morning, last afternoon, last evening, last night, fill the previous day and today in list_date.
When users ask questions related to time periods such as last month, last hours, last days, or last week, try your best to fill in list_date and list_time with full dates and times within the specified range.

<|eot_id|><|start_header_id|>user<|end_header_id|>

Provide me the respiratory rate and systolic pressure of the patient 05419 in the last 19 days.<|eot_id|><|start_header_id|>assistant<|end_header_id|> """

In [25]:
inputs = processor(None, prompt, return_tensors="pt").to(model.device)

output = model.generate(**inputs, max_new_tokens=1000, early_stopping=True)
full_response = processor.decode(output[0])
text_without_prompt = full_response[len(prompt):]

/home/thanh.ho/.conda/envs/taco/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:676: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


In [27]:
print(text_without_prompt)

er_id|>assistant<|end_header_id|>

<|python_tag|>{
  "patient_id": "05419",
  "list_date": [
    "2023-12-27",
    "2023-12-28",
    "2023-12-29",
    "2023-12-30",
    "2023-12-31",
    "2024-01-01",
    "2024-01-02",
    "2024-01-03",
    "2024-01-04",
    "2024-01-05",
    "2024-01-06",
    "2024-01-07",
    "2024-01-08",
    "2024-01-09",
    "2024-01-10",
    "2024-01-11",
    "2024-01-12",
    "2024-01-13",
    "2024-01-14"
  ],
  "list_time": [],
  "vital_sign": [
    "respiratory_rate",
    "systolic_pressure"
  ],
  "is_plot": false,
  "recognition": false,
  "is_image": false
}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

I am not able to provide a response to that request.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{
  "patient_id": "05419",
  "list_date": [
    "2023-12-27",
    "2023-12-28",
    "2023-12-29",
    "2023-12-30",
    "2023-12-31",
    "2024-01-01",
    "2024-01-02",
    "2024-01-03",
    "2024-01-04",
    "2024-01-05",
    "2024-01-06"

In [32]:
from transformers import pipeline
import torch

model_id = "meta-llama/Llama-3.2-11B-Vision"
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

messages = [
    {"role": "user", "content": """
    <|start_header_id|>system<|end_header_id|>
    The current time is 2024-01-15 03:47:43.

    You are a helpful assistant. Your task is to detect the user's intent and provide a response in the form of a JSON object complete with the following keys:
    1. 'patient_id': A string representing the ID of the patient the user is inquiring about. If patient ID is not provided, leave it empty.
    2. 'list_date': A list of dates for which data needs to be retrieved to answer the user's question in format of yyyy-mm-dd. Leave the list empty if the user asks for data right now.
    3. 'list_time':  A list of times for which data needs to be retrieved to answer the user's question in format of hh:mm:ss. The system saves data in 30-minute period like 00:00:00 and 00:30:00. Leave the list empty if the user asks for data right now.
    4. 'vital_sign': A list of vital signs that the user is asking for. Here are the available vital sign: heart_rate, systolic_pressure, diastolic_pressure, respiratory_rate, body_temperature, oxygen_saturation
    5. 'is_plot': A Boolean value indicating whether the system needs to generate a plot to answer the question more clearly (when the number of data points is too large) or if the user has requested a plot.
    6. 'recognition': A Boolean value indicating whether the user is asking for activity or emotion recognition of the patient. Set to true if the user is asking for this information, otherwise false.
    7. 'is_image': A Boolean value indicating whether the user is asking to show an image of the patient.

    If the user provides the time in both AM and PM formats, you can still interpret it in the 'hh:mm:ss' format.
    If the user asks for sessions during the day, please use the following information to fill in list_time: Morning is from 05:00:00 (5am) to 12:00:00 (12pm), Afternoon is from 12:00:00 (12pm) to 17:00:00 (5pm), Evening is from 17:00:00 (5pm) to 21:00:00 (9pm), Night is from 21:00:00 (9pm) to 04:00:00 (4am) the following day.
    It's important to remember that health status consists of vital signs.
    When the user asks for data from the last a number of hours, today should also be included in the list_date.
    When the user asks for data last morning, last afternoon, last evening, last night, fill the previous day and today in list_date.
    When users ask questions related to time periods such as last month, last hours, last days, or last week, try your best to fill in list_date and list_time with full dates and times within the specified range.

    <|eot_id|><|start_header_id|>user<|end_header_id|>

    Provide me the respiratory rate and systolic pressure of the patient 05419 in the last 19 days.<|eot_id|><|start_header_id|>assistant<|end_header_id|> """
    }]
outputs = pipe(
    messages,
    max_new_tokens=1000,
)
response = outputs[0]["generated_text"][-1]["content"]
print(response)
# Arrrr, me hearty! Yer lookin' fer a bit o' information about meself, eh? Alright then, matey! I be a language-generatin' swashbuckler, a digital buccaneer with a penchant fer spinnin' words into gold doubloons o' knowledge! Me name be... (dramatic pause)...Assistant! Aye, that be me name, and I be here to help ye navigate the seven seas o' questions and find the hidden treasure o' answers! So hoist the sails and set course fer adventure, me hearty! What be yer first question?



Loading checkpoint shards: 100%|██████████| 5/5 [00:01<00:00,  4.60it/s]
Device set to use cpu


AttributeError: 'NoneType' object has no attribute 'apply_chat_template'